In [2]:
import pandas as pd

In [10]:
df = pd.read_csv('data.phot', delim_whitespace=True, header=None)


In [11]:
df = df[[2,3,16,29]]
df.columns=['x','y','V','I']
df['V-I']=df['V']-df['I']

In [12]:
    df.head(5)

,x,y,V,I,V-I
0,2388.82,1804.24,28.052,26.850,1.202
1,2926.23,875.59,27.733,26.751,0.982
2,3233.50,120.59,27.793,26.530,1.263
3,3413.04,1944.85,28.147,26.900,1.247
4,2464.21,1444.66,28.125,26.703,1.422


In [14]:
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.charts import Scatter
from bokeh.io import output_notebook
output_notebook()

s1 = ColumnDataSource(data=df)
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6, size=0.1)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=[-2,2], y_range=[30,26],
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6, size=0.1)

s1.callback = CustomJS(args=dict(s2=s2), code="""
        var inds = cb_obj.selected['1d'].indices;
        var d1 = cb_obj.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (i = 0; i < inds.length; i++) {
            d2['x'].push(d1['V-I'][inds[i]])
            d2['y'].push(d1['I'][inds[i]])
        }
        s2.trigger("change");
        p1.title="Selected";
    """)

layout = row(p1, p2)

show(layout)

Loading BokehJS ...

In [15]:
p3 = figure()
p3.circle('V-I','I', source=s1, size=0.1)

GlyphRenderer(id='1e695515-428a-4468-be57-fdf26603b007', ...)

In [17]:
show(p3)